In [1]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.nn.utils.prune as prune
import torch.optim as optim
import time
from easydict import EasyDict

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
import utils
import prune_utils
from run import run
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
torch.cuda.is_available()
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')# Load Dataset

# Load Dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

CIFAR100train = torchvision.datasets.CIFAR100(root="./", train=True, download=True, transform=transform)
CIFAR100test = torchvision.datasets.CIFAR100(root="./", train=False, download=True, transform=transform)

CIFAR100training_set = [(CIFAR100train[t][0].to(device), CIFAR100train[t][1]) for t in range(len(CIFAR100train))]
CIFAR100test_set = [(CIFAR100test[t][0].to(device), CIFAR100test[t][1]) for t in range(len(CIFAR100test))]

Files already downloaded and verified
Files already downloaded and verified


In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)])

MNISTtrain = torchvision.datasets.MNIST(root="./", train=True, download=True, transform=transform)
MNISTtest = torchvision.datasets.MNIST(root="./", train=False, download=True, transform=transform)

MNISTtraining_set = [(MNISTtrain[t][0].to(device), MNISTtrain[t][1]) for t in range(len(MNISTtrain))]
MNISTtest_set = [(MNISTtest[t][0].to(device), MNISTtest[t][1]) for t in range(len(MNISTtest))]

In [7]:
batch_size = 1024

CIFAR100trainloader = torch.utils.data.DataLoader(CIFAR100training_set, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
CIFAR100testloader = torch.utils.data.DataLoader(CIFAR100test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=0)
MNISTtrainloader = t# MNIST ResNet-18 No-Finetuningorch.utils.data.DataLoader(MNISTtraining_set, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
MNISTtestloader = torch.utils.data.DataLoader(MNISTtest_set, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

# Sample Usage

In [ ]:
##################################################################
## No Finetune
##################################################################

cfg.epochs = 20
cfg.log_interval = 5
cfg.l2_regularization = 1e-6
cfg.lr = 0.001
cfg.train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=2),
    transforms.RandomHorizontalFlip(),
])

cfg.model = 'vgg19' # resnet18, resnet34, resnet50, vgg16, vgg19
cfg.dataset = 'cifar100' # mnist, cifar100
cfg.prune_strategy = 'l1' # l1, l2unstructured, l1structured, random

cfg.trainloader = CIFAR100trainloader # CIFAR100trainloader, MNISTtrainloader
cfg.testloader = CIFAR100testloader # CIFAR100testloader, MNISTtestloader

cfg.finetune = False
cfg.load_model = True
cfg.model_path = f'models/{cfg.model}_{cfg.dataset}.pth'


cfg.prune_layers = 'fc'

cfg.file_name = f'results/{cfg.dataset}_{cfg.model}_{cfg.prune_strategy}_{cfg.prune_layers}.txt'

run(cfg, device)

##################################################################
## Finetune
##################################################################

cfg.epochs = 20
cfg.log_interval = 5
cfg.l2_regularization = 1e-6
cfg.lr = 0.0001
cfg.train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=2),
    transforms.RandomHorizontalFlip(),
])

cfg.finetune = True
cfg.load_model = True
cfg.file_name = f'results/{cfg.dataset}_{cfg.model}_{cfg.prune_strategy}_{cfg.prune_layers}_finetune.txt'

run(cfg, device)